In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import torch
from tqdm.notebook import tqdm

In [2]:
def compute_iou(annotation, mask):
    """Annotation: ground truth (512, 512), Mask: prediction (512, 512)"""

    # Compute intersection
    intersection = np.sum(np.logical_and(annotation, mask))

    # Compute union
    union = np.sum(np.logical_or(annotation, mask))

    # Compute intersection over union
    iou_score = intersection / union

    return intersection, union, iou_score


def interpolate(heatmap: np.ndarray, size=(512, 512), mode="bilinear"):
    """Interpolate heatmap to match the size of the ground truth"""

    # Convert to torch tensor
    heatmap = torch.from_numpy(heatmap)
    # Add batch and channel dimension
    heatmap = heatmap.unsqueeze(0)
    if heatmap.ndim != 4:
        heatmap = heatmap.unsqueeze(0)
    # Interpolate
    heatmap = torch.nn.functional.interpolate(heatmap, size=size, mode=mode)
    # Convert back to numpy
    heatmap = heatmap.squeeze().squeeze().numpy()

    return heatmap

In [3]:
import denseCRF
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys

def densecrf(I, P):
    """
    input parameters:
        I    : a numpy array of shape [H, W, C], where C should be 3.
               type of I should be np.uint8, and the values are in [0, 255]
        P    : a probability map of shape [H, W, L], where L is the number of classes
               type of P should be np.float32
        param: a tuple giving parameters of CRF (w1, alpha, beta, w2, gamma, it), where
                w1    :   weight of bilateral term, e.g. 10.0
                alpha :   spatial distance std, e.g., 80
                beta  :   rgb value std, e.g., 15
                w2    :   weight of spatial term, e.g., 3.0
                gamma :   spatial distance std for spatial term, e.g., 3
                it    :   iteration number, e.g., 5
    output parameters:
        out  : a numpy array of shape [H, W], where pixel values represent class indices. 
    """
    w1    = 10.0  # weight of bilateral term
    alpha = 80    # spatial std
    beta  = 13    # rgb  std
    w2    = 3.0   # weight of spatial term
    gamma = 3     # spatial std
    it    = 5.0   # iteration
    param = (w1, alpha, beta, w2, gamma, it)
    out = denseCRF.densecrf(I, P, param) 
    return out   

## OVAM

## VOC-sim

In [ ]:
dataset_path = Path('voc_sim')

annotations_folder = Path('../ovam/voc_sim/annotations')
images_folder = Path('../ovam/voc_sim/images')
sa_folder =  Path('../ovam_sa/voc_sim/sa')


def load_heatmap(classname, seed, res16, res32, res64, opt):
        hs = []
        opt_suffix = "" if not opt else "_opt"
        if res16:
            filename = dataset_path / f"{classname}_{seed.replace('seed', '')}_a16{opt_suffix}.npy"
            h = np.load(filename)
            # INterpolate to 64x64
            h = interpolate(h, size=(64, 64))
            hs.append(h)
        if res32:
            filename = dataset_path / f"{classname}_{seed.replace('seed', '')}_a32{opt_suffix}.npy"
            h = np.load(filename)
            # INterpolate to 64x64
            h = interpolate(h, size=(64, 64))
            hs.append(h)
        if res64:
            filename = dataset_path / f"{classname}_{seed.replace('seed', '')}_a64{opt_suffix}.npy"
            h = np.load(filename)
            hs.append(h)
        heatmap = np.stack(hs, axis=0)
        heatmap = heatmap.sum(axis=0)
        return heatmap



apply_crf = True
suffix_crf = "_crf" if apply_crf else ""
min_sa = 0.85
min_sa_optimized = 0.95

re_results = []
res16, res32, res64 = True, True, True
# Eight combinations of res16, res32, res64
for apply_sa, apply_crf in [(False, False), (True, False), (False, True), (True, True)]:
    if apply_sa:
        # Hyperparameters without optimization
        threshold = 0.4 # Threshold used in the paper DAAM
        threshold_optimized = 0.75
    else:
        threshold = 0.4 # Threshold used in the paper DAAM
        threshold_optimized = 0.8

    # Iterate throuth annotations
    results = []
    
    for annotation_path in tqdm(list(annotations_folder.iterdir())):
        example_result_dict = {}
        classname, model, seed, _ = annotation_path.stem.split('_')
        image_path = images_folder / f"{classname}_{model}_{seed}.png"
        sa_path = sa_folder / f"sa15_{classname}_{seed.replace('seed', '')}_sa.npy"
        heatmap = load_heatmap(classname, seed, res16, res32, res64, opt=False)
        heatmap_optimized = load_heatmap(classname, seed, res16, res32, res64, opt=True)
        
        # Check all exists
        assert annotation_path.exists(), f"Annotation {annotation_path} does not exist"
        assert image_path.exists(), f"Image {image_path} does not exist"
        assert sa_path.exists(), f"SA {sa_path} does not exist"
        assert heatmap.shape[1:] == (64, 64)
        assert heatmap.shape[0] < 10
        assert heatmap_optimized.shape[1:] == (64, 64)
        assert heatmap_optimized.shape[0] == 2

        
        example_result_dict['classname'] = classname
        example_result_dict['model'] = model
        example_result_dict['seed'] = seed
        example_result_dict['image_path'] = image_path.name
        example_result_dict['annotation_path'] = annotation_path.name
        #example_result_dict['heatmap_path'] = heatmap_path.name
        #example_result_dict['heatmap_optimized_path'] = heatmap_optimized_path.name
        

        # Load image
        image = np.array(Image.open(image_path))
        assert image.shape == (512, 512, 3), f"Image {image_path} has wrong shape {image.shape}"
        
        # Load annotation. Convert in binary mask
        annotation = np.array(Image.open(annotation_path))
        assert annotation.shape == (512, 512, 3), f"Annotation {annotation_path} has wrong shape {annotation.shape}"
        annotation = annotation.sum(axis=-1) != 0
        assert annotation.shape == (512, 512), f"Annotation aggregated {annotation_path} has wrong shape {annotation.shape}"
        
        
        # Load Self-Attention map
        sa = np.load(sa_path)
        assert sa.shape == (64, 64)
        sa = interpolate(sa, size=(512, 512))
        assert sa.shape == (512, 512)


        # Reescale self-attention to [min_sa, 1]
        sa = (sa - sa.min()) / (sa.max() - sa.min())
        sa = sa * (1 - min_sa) + min_sa
        assert abs(sa.min()- min_sa) < 0.001, f"Min sa {sa.min()} is not {min_sa}"
        assert abs(sa.max()- 1) < 0.001, f"Max sa {sa.max()} is not 1"
        
        # Binarize using DAAM procedure
        heatmap = interpolate(heatmap[-2], size=(512, 512))
        assert heatmap.shape == (512, 512), f"Heatmap {heatmap_path} has wrong shape {heatmap.shape}"
        heatmap = heatmap / heatmap.max()
        if apply_sa:
            heatmap = heatmap * sa # Apply sa
        mask = heatmap > threshold

        if apply_crf:
            mask = np.stack([1 - mask, mask], axis=-1).astype(np.float32)
            mask = densecrf(image, mask)

        assert mask.shape == (512, 512), f"Mask {mask} has wrong shape {mask.shape}"

        i_normal, u_normal, iou_normal = compute_iou(annotation=annotation, mask=mask)
        example_result_dict['iou_normal'] = iou_normal
        example_result_dict['i_normal'] = i_normal
        example_result_dict['u_normal'] = u_normal


        
        # Load mask (optimized)
        heatmap_optimized = heatmap_optimized[1] # We stored in 0 the background and in 1 the token related to the foreground object
        assert heatmap_optimized.shape == (64, 64), f"Heatmap {heatmap_optimized_path} has wrong shape {heatmap_optimized.shape}"
        heatmap_optimized = interpolate(heatmap_optimized, size=(512, 512))
        assert heatmap_optimized.shape == (512, 512), f"Heatmap {heatmap_optimized_path} has wrong shape {heatmap_optimized.shape}"


        # Reescale self-attention to [min_sa_optimized, 1]
        sa = (sa - sa.min()) / (sa.max() - sa.min())
        sa = sa * (1 - min_sa_optimized) + min_sa_optimized
        assert abs(sa.min()- min_sa_optimized) < 0.001, f"Min sa {sa.min()} is not {min_sa}"
        assert abs(sa.max()- 1) < 0.001, f"Max sa {sa.max()} is not 1"

        # Binarize using DAAM procedure
        heatmap_optimized = heatmap_optimized / heatmap_optimized.max()
        if apply_sa:
            heatmap_optimized = heatmap_optimized * sa
        mask_optimized = heatmap_optimized > threshold_optimized
        if apply_crf:
            mask_optimized = np.stack([1 - mask_optimized, mask_optimized], axis=-1).astype(np.float32)
            mask_optimized = densecrf(image, mask_optimized)

        assert mask_optimized.shape == (512, 512), f"Mask {mask_optimized} has wrong shape {mask_optimized.shape}"

        i_optimized, u_optimized, iou_optimized = compute_iou(annotation=annotation, mask=mask_optimized)
        example_result_dict['iou_optimized'] = iou_optimized
        example_result_dict['i_optimized'] = i_optimized
        example_result_dict['u_optimized'] = u_optimized

        results.append(example_result_dict)
        
        
    # Aggregated by example
    df_results = pd.DataFrame(results)
    df_results['experiment'] = "voc-sim - daam"
    # df_results.to_csv(dataset_path / f'ovam_voc_sim_results_sa{suffix_crf}.csv', index=False)


    # Aggregated results by class
    df_classes = df_results.groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
    df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
    df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
    df_classes['experiment'] = "voc-sim - ovam"
    df_classes = df_classes.sort_values('classname').reset_index(drop=True)
    # df_classes.to_csv(dataset_path / f'ovam_voc_sim_class_results_sa{suffix_crf}.csv', index=False)

    df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                            'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

    df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
    df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
    df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
    df_overall['experiment'] = "voc-sim - ovam"

    #df_overall.to_csv(dataset_path / f'ovam_voc_sim_overall_results_sa{suffix_crf}.csv', index=False)

    df_overall_display = df_overall[["miou_normal","iou_overall_normal", "miou_optimized",  "iou_overall_optimized"]]
    df_overall_display = (100*df_overall_display).round(1)
    df_overall_display["dcrf"] = apply_crf
    df_overall_display["sa"] = apply_sa

    display(df_overall_display)

    re_results.append(df_overall_display)

df_reresults = pd.concat(re_results)
df_reresults



# COCO-cap

In [ ]:
dataset_path = Path('coco_cap')

annotations_folder = Path('../ovam/coco_captions/annotations')
images_folder = Path('../ovam/coco_captions/images')
sa_folder =  Path('../ovam_sa/coco_cap/sa')


def load_heatmap(classname, seed,  caption, res16, res32, res64, opt):
        hs = []
        opt_suffix = "" if not opt else "_opt"
        if res16:
            filename = dataset_path / f"{classname.replace(' ', '-')}_{caption}_{seed.replace('seed', '')}_a16{opt_suffix}.npy"
            h = np.load(filename)
            # INterpolate to 64x64
            h = interpolate(h, size=(64, 64))
            hs.append(h)
        if res32:
            filename = dataset_path / f"{classname.replace(' ', '-')}_{caption}_{seed.replace('seed', '')}_a32{opt_suffix}.npy"
            h = np.load(filename)
            # INterpolate to 64x64
            h = interpolate(h, size=(64, 64))
            hs.append(h)
        if res64:
            filename = dataset_path / f"{classname.replace(' ', '-')}_{caption}_{seed.replace('seed', '')}_a64{opt_suffix}.npy"
            h = np.load(filename)
            hs.append(h)
        heatmap = np.stack(hs, axis=0)
        heatmap = heatmap.sum(axis=0)
        return heatmap

# Hyperparameters

# Hyperparameters without optimization
threshold = .4 #0.5 # Threshold used in the paper DAAM
min_sa = .75 # 0.91

# Threshold found empirically using the training set employed for optimizng the tokens
threshold_optimized = 0.8
min_sa_optimized = .75 #0.99


apply_crf = True
suffix_crf = "_crf" if apply_crf else ""

re_results = []
heatmap_path = None
heatmap_optimized_path = None

res16, res32, res64 = True, True, True
# Eight combinations of res16, res32, res64

min_sa = 0.85
min_sa_optimized = 0.95

for apply_sa, apply_crf in [(False, False), (True, False), (False, True), (True, True)]:
    if apply_sa:
        # Hyperparameters without optimization
        threshold = 0.4 # Threshold used in the paper DAAM
        threshold_optimized = 0.8
    else:
        threshold = 0.4 # Threshold used in the paper DAAM
        threshold_optimized = 0.75



    # Iterate throuth annotations
    results = []
    for annotation_path in tqdm(list(annotations_folder.iterdir())):
        example_result_dict = {}
        classname, model, _, caption, _, seed = annotation_path.stem.split('_')
        model = model.replace('-', '')
        caption = f"caption{caption}"
        seed = f"seed{seed}"
        
        image_path = images_folder / f"{classname}_{model}_{caption}_{seed}.png"
        sa_path = sa_folder / f"sd15attn2mask_{classname.replace(' ', '-')}_{caption.replace('caption', '')}_{seed.replace('seed', '')}_sa.npy"


        heatmap = load_heatmap(classname, seed, caption, res16, res32, res64, opt=False)
        heatmap_optimized = load_heatmap(classname, seed, caption, res16, res32, res64, opt=True)
        #print(heatmap.shape)
        assert annotation_path.exists(), f"Annotation {annotation_path} does not exist"
        assert image_path.exists(), f"Image {image_path} does not exist"
        assert sa_path.exists(), f"Mask {sa_path} does not exist"
        

        # Add paths to result dict
        example_result_dict['classname'] = classname
        example_result_dict['model'] = model
        example_result_dict['seed'] = seed
        
        example_result_dict['image_path'] = image_path.name
        example_result_dict['annotation_path'] = annotation_path.name
        #example_result_dict['heatmap_path'] = heatmap_path.name
        #example_result_dict['heatmap_optimized_path'] = heatmap_optimized_path.name

        # Get info of coco caption used using caption_id
        caption_id = int(caption.replace('caption', ''))
        # row = df_coco_captions.query("caption_id==@caption_id")
        # assert len(row) == 1, f"Caption {caption_id} not found in df_coco_captions"
        # row = row.iloc[0]
        # prompt = row['caption']
        # word_included = row['word_included']
        # coco_categories = row['categories']


        # Add info to results
        example_result_dict['coco_caption_id'] = caption_id
        #example_result_dict['prompt'] = prompt
        #example_result_dict['word_included'] = word_included
        #example_result_dict['coco_categories'] = coco_categories

        # Load image
        image = np.array(Image.open(image_path))
        assert image.shape == (512, 512, 3), f"Image {image_path} has wrong shape {image.shape}"

        # Load SA
        sa = np.load(sa_path)
        assert sa.shape == (64, 64), f"SA {sa_path} has wrong shape {sa.shape}"
        sa = interpolate(sa, size=(512, 512))
        assert sa.shape == (512, 512), f"SA {sa_path} has wrong shape {sa.shape}"


        # Reescale self-attention to [min_sa, 1]
        sa = (sa - sa.min()) / (sa.max() - sa.min())
        sa = sa * (1 - min_sa) + min_sa
        assert abs(sa.min()- min_sa) < 0.001, f"Min sa {sa.min()} is not {min_sa}"
        assert abs(sa.max()- 1) < 0.001, f"Max sa {sa.max()} is not 1"


        # Load annotation. Convert in binary mask
        annotation = np.array(Image.open(annotation_path))
        
        assert annotation.shape == (512, 512, 3), f"Annotation {annotation_path} has wrong shape {annotation.shape}"
        annotation = annotation.sum(axis=-1) != 0
        assert annotation.shape == (512, 512), f"Annotation aggregated {annotation_path} has wrong shape {annotation.shape}"
        
        # Load heatmap
        heatmap = heatmap[-2] # We stored in 0 the background and in 1 the token related to the foreground object
        assert heatmap.shape == (64, 64), f"Heatmap {heatmap_path} has wrong shape {heatmap.shape}"
        heatmap = interpolate(heatmap, size=(512, 512), mode="bicubic")
        assert heatmap.shape == (512, 512), f"Heatmap {heatmap_path} has wrong shape {heatmap.shape}"
        
        # Binarize using DAAM procedure + dCRF
        heatmap = heatmap / heatmap.max()
        if apply_sa:
            heatmap = heatmap * sa
        mask = heatmap > threshold

        if apply_crf:
            mask = np.stack([1 - mask, mask], axis=-1).astype(np.float32)
            mask = densecrf(image, mask)

        i_normal, u_normal, iou_normal = compute_iou(annotation=annotation, mask=mask)
        example_result_dict['iou_normal'] = iou_normal
        example_result_dict['i_normal'] = i_normal
        example_result_dict['u_normal'] = u_normal

        # Load mask (optimized)
        heatmap_optimized = heatmap_optimized[1] # We stored in 0 the background and in 1 the token related to the foreground object
        assert heatmap_optimized.shape == (64, 64), f"Heatmap {heatmap_optimized_path} has wrong shape {heatmap_optimized.shape}"
        heatmap_optimized = interpolate(heatmap_optimized, size=(512, 512), mode="bilinear")
        assert heatmap_optimized.shape == (512, 512), f"Heatmap {heatmap_optimized_path} has wrong shape {heatmap_optimized.shape}"
        
        # Rescale sa
        sa = (sa - sa.min()) / (sa.max() - sa.min())
        sa = sa * (1 - min_sa_optimized) + min_sa_optimized
        assert abs(sa.min()- min_sa_optimized) < 0.001, f"Min sa {sa.min()} is not {min_sa}"
        assert abs(sa.max()- 1) < 0.001, f"Max sa {sa.max()} is not 1"

        # Binarize using DAAM procedure + dCRF
        heatmap_optimized = heatmap_optimized / heatmap_optimized.max()
        if apply_sa:
            heatmap_optimized = heatmap_optimized * sa
        mask_optimized = heatmap_optimized > threshold_optimized
        if apply_crf:
            mask_optimized = np.stack([1 - mask_optimized, mask_optimized], axis=-1).astype(np.float32)
            mask_optimized = densecrf(image, mask_optimized)

        i_optimized, u_optimized, iou_optimized = compute_iou(annotation=annotation, mask=mask_optimized)
        example_result_dict['iou_optimized'] = iou_optimized
        example_result_dict['i_optimized'] = i_optimized
        example_result_dict['u_optimized'] = u_optimized

        results.append(example_result_dict)    

    df_results = pd.DataFrame(results)
    df_results['experiment'] = "coco-cap - ovam"
    #df_results.to_csv(dataset_path / f'ovam_coco_captions_{suffix_crf}results.csv', index=False)

    # All results (included and not included)
    assert dataset_path.name == 'coco_cap', f"Dataset path {dataset_path} is not coco_captions"

    # Aggregated results by class
    df_classes = df_results.groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
    df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
    df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
    df_classes['experiment'] = "voc-sim - ovam"
    df_classes = df_classes.sort_values('classname').reset_index(drop=True)
    #df_classes.to_csv(dataset_path / f'ovam-cap_class_results_all{suffix_crf}.csv', index=False)

    df_classes_display = df_classes[["classname", 'iou_normal', 'iou_optimized']].copy()
    df_classes_display['iou_normal'] = (100*df_classes_display['iou_normal']).round(1)
    df_classes_display['iou_optimized'] = (100*df_classes_display['iou_optimized']).round(1)
    #df_classes_display.T.to_excel(dataset_path / f'ovam-cap_class_results_all{suffix_crf}.xlsx', index=False)

    # display(df_classes_display.T)


    # Aggregate overall results
    df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                        'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

    df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
    df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
    df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
    df_overall['experiment'] = "coco-cap - grounded diffusion"
    #df_overall.to_csv(dataset_path / f'ovam-cap_overall_results_all{suffix_crf}.csv', index=False)
    df_overall_display = df_overall[['iou_overall_normal', 'miou_normal', 'iou_overall_optimized', 'miou_optimized']].copy()
    df_overall_display = (100*df_overall_display).round(1)
    df_overall_display["dcrf"] = apply_crf
    df_overall_display["sa"] = apply_sa
    display(df_overall_display[["miou_normal","iou_overall_normal", "miou_optimized",  "iou_overall_optimized", "sa", "dcrf"]])

    re_results.append(df_overall_display)

df_reresults = pd.concat(re_results)
df_reresults


In [ ]:

assert dataset_path.name == 'coco_cap', f"Dataset path {dataset_path} is not coco_captions"

for included in [True, False]:
    included_str = 'included' if included else 'non_included'
    print(included_str)
    # Aggregated results by class
    df_classes = df_results.query("word_included==@included").groupby(['classname', 'model']).aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum'}).reset_index()
    df_classes['iou_normal'] = df_classes['i_normal'] / df_classes['u_normal']
    df_classes['iou_optimized'] = df_classes['i_optimized'] / df_classes['u_optimized']
    df_classes['experiment'] = "voc-sim - grounded diffusion"
    df_classes = df_classes.sort_values('classname').reset_index(drop=True)
    #df_classes.to_csv(dataset_path / 'grounded_diffusion_coco-cap_class_results_all.csv', index=False)

    df_classes_display = df_classes[["classname", 'iou_normal', 'iou_optimized']].copy()
    df_classes_display['iou_normal'] = (100*df_classes_display['iou_normal']).round(1)
    df_classes_display['iou_optimized'] = (100*df_classes_display['iou_optimized']).round(1)
    df_classes_display.T.to_excel(dataset_path / f'ovam-cap_class_results_{included_str}_{suffix_crf}.xlsx', index=False)

    display(df_classes_display.T)

    # Aggregate overall results
    df_overall = df_classes.groupby('model').aggregate({'i_normal': 'sum', 'u_normal': 'sum', 'i_optimized': 'sum', 'u_optimized': 'sum',
                                        'iou_normal': 'mean', 'iou_optimized': 'mean'}).reset_index()

    df_overall.rename(columns={'iou_normal': 'miou_normal', 'iou_optimized': 'miou_optimized'}, inplace=True)
    df_overall['iou_overall_normal'] = df_overall['i_normal'] / df_overall['u_normal']
    df_overall['iou_overall_optimized'] = df_overall['i_optimized'] / df_overall['u_optimized']
    df_overall['experiment'] = "coco-cap - grounded diffusion"
    df_overall.to_csv(dataset_path / f'ovam_coco-cap_overall_results_{included_str}_{suffix_crf}.csv', index=False)

    df_overall_display = df_overall[['miou_normal', 'iou_overall_normal',  'miou_optimized', 'iou_overall_optimized']].copy()
    df_overall_display = (100*df_overall_display).round(1)
    display(df_overall_display)

df_reresults = pd.concat(re_results)
df_reresults
